In [1]:
import os

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["HF_HOME"] = "/root/autodl-tmp/cache/"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["TMPDIR"]="/root/autodl-tmp/tmp"
os.environ["TORCH_HOME"]="/root/autodl-tmp/torch"
os.environ["PYTHONDONTWRITEBYTECODE"]="1"


import json
import re
from pathlib import Path

import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from transformers import AutoProcessor, LlavaNextForConditionalGeneration
from peft import PeftModel
from tqdm import tqdm

# ---- Config ----
CATEGORY = "Automotive"
CANDIDATE_TYPE = "candidates_st"
BASE_MODEL_NAME = "llava-hf/llava-v1.6-mistral-7b-hf"
VISION_ADAPTER_DIR = f"./out_distilled/{CATEGORY}/vision_lora_adapter_best"
LM_ADAPTER_DIR = f"./out_finetuned/{CATEGORY}/prompt_tuning/trained_lora_adapter"
FINETUNE_OUTPUT_DIR = f"./test_out/{CATEGORY}"
#DATA_ROOT = Path("../data")
DATA_ROOT = Path("/root/autodl-tmp/lavic/data")
ITEM_META_PATH = DATA_ROOT / f"item2meta_train_{CATEGORY}.with_desc.json"
IMAGE_DIR = DATA_ROOT / "train_images" / CATEGORY
TEST_DATA_PATH = DATA_ROOT / CATEGORY / "test.jsonl"
BATCH_SIZE = 1
MAX_LENGTH = 2048
DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"
DTYPE = torch.float16 if DEVICE == "cuda:1" else torch.float32

print("BASE_MODEL_NAME:", BASE_MODEL_NAME)
print("VISION_ADAPTER_DIR:", VISION_ADAPTER_DIR)
print("LM_ADAPTER_DIR:", LM_ADAPTER_DIR)
print("ITEM_META_PATH:", ITEM_META_PATH)
print("IMAGE_DIR:", IMAGE_DIR)
print("TEST_DATA_PATH:", TEST_DATA_PATH)

IMAGE_TOKENS = [
    "<ItemImageEmb1>", "<ItemImageEmb2>", "<ItemImageEmb3>",
    "<ItemImageEmb4>", "<ItemImageEmb5>"
]

BASE_MODEL_NAME: llava-hf/llava-v1.6-mistral-7b-hf
VISION_ADAPTER_DIR: ./out_distilled/Automotive/vision_lora_adapter_best
LM_ADAPTER_DIR: ./out_finetuned/Automotive/prompt_tuning/trained_lora_adapter
ITEM_META_PATH: /root/autodl-tmp/lavic/data/item2meta_train_Automotive.with_desc.json
IMAGE_DIR: /root/autodl-tmp/lavic/data/train_images/Automotive
TEST_DATA_PATH: /root/autodl-tmp/lavic/data/Automotive/test.jsonl


In [2]:
from pathlib import Path
print("cwd:", Path.cwd())
print("ITEM_META_PATH:", ITEM_META_PATH)
print("resolved:", ITEM_META_PATH.resolve())
print("exists:", ITEM_META_PATH.exists())

DATA_ROOT = Path("/root/autodl-tmp/lavic/data")

cwd: /root/autodl-tmp/lavic
ITEM_META_PATH: /root/autodl-tmp/lavic/data/item2meta_train_Automotive.with_desc.json
resolved: /root/autodl-tmp/lavic/data/item2meta_train_Automotive.with_desc.json
exists: True


In [3]:
def load_item_meta(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def load_jsonl(file_path):
    rows = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            rows.append(json.loads(line))
    return rows

def evaluate_recall_at_k(recommended_ids, gt_items, k=1):
    hits = 0
    total = 0
    for rec_id, gts in zip(recommended_ids, gt_items):
        for gt in gts:
            if rec_id is not None and rec_id == gt:
                hits += 1
            total += 1
    recall = hits / total if total > 0 else 0.0
    return recall

def check_validity(file_path, model_key):
    candidates_key = f"candidates_{model_key}"
    recommended_key = f"recommended_{model_key}"
    total = 0
    valid = 0
    invalid_entries = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for idx, line in enumerate(f, 1):
            try:
                data = json.loads(line.strip())
            except json.JSONDecodeError:
                continue
            recommended = data.get(recommended_key, None)
            candidates = data.get(candidates_key, [])
            total += 1
            if recommended in candidates:
                valid += 1
            else:
                invalid_entries.append({
                    "line_number": idx,
                    "recommended_id": recommended,
                    "candidates": candidates
                })
    validity = valid / total if total > 0 else 0.0
    return validity, invalid_entries, total

def prepare_candidate_info(candidates, item_meta, image_dir, default_image):
    candidate_info = []
    for cid in candidates:
        title = item_meta.get(cid, {}).get('title', 'No Title')
        image_path = image_dir / f"{cid}_0.jpg"
        if image_path.exists():
            image = Image.open(image_path).convert('RGB')
        else:
            image = default_image
        candidate_info.append({
            'id': cid,
            'title': title,
            'image': image
        })
    return candidate_info

CONV_CHARS_PER_TOKEN = 4

def build_prompt(conversation_text, candidates_info, max_conv_chars=None):
    if max_conv_chars is not None and len(conversation_text) > max_conv_chars:
        head_len = max_conv_chars // 2
        tail_len = max_conv_chars - head_len
        conversation_text = (
            conversation_text[:head_len]
            + "\n...[TRUNCATED]...\n"
            + conversation_text[-tail_len:]
        )
    prompt = (
        "You are an AI assistant specialized in providing personalized product recommendations based on user conversations. "
        "You are given a conversation between a user seeking recommendation (denoted by <submission>) and other users providing comments (denoted by <comment>). "
        "You are also given a set of candidate products with their IDs, titles and images formatted as \"ID: title\" followed by an image. "
        "Among the candidates, recommend the most relevant product to the seeker. "
        "Only reply with its ID, and don't say anything else.\n\n"
        f"Conversation:\n{conversation_text}\n\n"
        "Candidates:\n"
    )
    for candidate in candidates_info:
        cid = candidate['id']
        title = candidate['title']
        prompt += f"{cid}: {title}\n"
        prompt += "".join(IMAGE_TOKENS) + "\n"
    prompt += "\nAssistant:"
    return prompt

class RecommendationDataset(Dataset):
    def __init__(self, data, item_meta, image_dir, candidate_type, max_conv_chars=None):
        self.data = data
        self.item_meta = item_meta
        self.image_dir = image_dir
        self.candidate_type = candidate_type
        self.default_image = Image.new('RGB', (336, 336), color=(255, 255, 255))
        if max_conv_chars is None:
            max_conv_chars = MAX_LENGTH * CONV_CHARS_PER_TOKEN
        self.max_conv_chars = max_conv_chars

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        entry = self.data[idx]
        conversation_text = entry.get('context', '')
        gt_items = entry.get('gt_items', [])
        candidates = entry.get(self.candidate_type, [])
        candidate_info = prepare_candidate_info(candidates, self.item_meta, self.image_dir, self.default_image)
        prompt = build_prompt(conversation_text, candidate_info, max_conv_chars=self.max_conv_chars)
        images = [c['image'] for c in candidate_info]
        return {
            'prompt': prompt,
            'images': images,
            'gt_items': gt_items,
            'entry_idx': idx
        }

class DataCollatorForLLaVA:
    def __init__(self, processor, tokenizer, max_length):
        self.processor = processor
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.image_token_ids = [
            self.tokenizer.convert_tokens_to_ids(tk) for tk in IMAGE_TOKENS
        ]

    def __call__(self, batch):
        prompts = [item['prompt'] for item in batch]
        images_per_sample = [item['images'] for item in batch]

        tokenized = self.tokenizer(
            prompts,
            add_special_tokens=True,
            padding=False,
            truncation=False,
        )

        input_ids_list = []
        for prompt_ids in tokenized['input_ids']:
            prompt_ids = list(prompt_ids)
            if len(prompt_ids) > self.max_length:
                bos_id = self.tokenizer.bos_token_id
                if bos_id is not None and prompt_ids[0] == bos_id:
                    prompt_ids = [bos_id] + prompt_ids[-(self.max_length - 1):]
                else:
                    prompt_ids = prompt_ids[-self.max_length:]
            input_ids_list.append(prompt_ids)

        max_len = max(len(ids) for ids in input_ids_list)
        pad_id = self.tokenizer.pad_token_id
        input_ids = torch.full((len(input_ids_list), max_len), pad_id, dtype=torch.long)
        attention_mask = torch.zeros((len(input_ids_list), max_len), dtype=torch.long)

        for i, ids in enumerate(input_ids_list):
            seq_len = len(ids)
            input_ids[i, :seq_len] = torch.tensor(ids, dtype=torch.long)
            attention_mask[i, :seq_len] = 1

        image_token_mask = torch.zeros_like(input_ids, dtype=torch.bool)
        for b_idx in range(input_ids.size(0)):
            for tid in self.image_token_ids:
                positions = (input_ids[b_idx] == tid).nonzero(as_tuple=False).squeeze(-1)
                image_token_mask[b_idx, positions] = True

        all_images = []
        for imgs in images_per_sample:
            all_images.extend(imgs)

        if all_images:
            images_processed = self.processor.image_processor(all_images, return_tensors='pt')
            images_tensor = images_processed['pixel_values']
        else:
            images_tensor = None

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'images': images_tensor,
            'image_token_mask': image_token_mask
        }


In [4]:
# Load base model + processor
base_model = LlavaNextForConditionalGeneration.from_pretrained(
    BASE_MODEL_NAME,
    torch_dtype=DTYPE,
    local_files_only=True,
).to(DEVICE)

processor = AutoProcessor.from_pretrained(BASE_MODEL_NAME, local_files_only=True)
tokenizer = processor.tokenizer

# Add special tokens
tokenizer.add_special_tokens({"additional_special_tokens": IMAGE_TOKENS})
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token": "[PAD]"})
tokenizer.pad_token = tokenizer.eos_token
base_model.resize_token_embeddings(len(tokenizer))

# Merge vision adapter
if VISION_ADAPTER_DIR and Path(VISION_ADAPTER_DIR).is_dir():
    base_model = PeftModel.from_pretrained(base_model, VISION_ADAPTER_DIR, local_files_only=True)
    base_model = base_model.merge_and_unload()
    base_model.to(DEVICE)

model = base_model
if LM_ADAPTER_DIR and Path(LM_ADAPTER_DIR).is_dir():
    model = PeftModel.from_pretrained(base_model, LM_ADAPTER_DIR, local_files_only=True)
    model.to(DEVICE)

model.eval()

# Data
item_meta = load_item_meta(ITEM_META_PATH)
test_data = load_jsonl(TEST_DATA_PATH)
test_ds = RecommendationDataset(test_data, item_meta, IMAGE_DIR, CANDIDATE_TYPE)
collator = DataCollatorForLLaVA(processor, tokenizer, MAX_LENGTH)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=lambda x: x)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [10]:
# Batch evaluate all categories with the last cell's logic
import os
from sentence_transformers import SentenceTransformer
import torch.nn as nn

CATEGORY_LIST = [
    "amazon_home",
    "amazon_fashion",
    "all_beauty",
    "Appliances",
    "Arts_Crafts_and_Sewing",
    "Automotive",
    "Baby_Products",
    "Books",
    "CDs_and_Vinyl",
    "Cell_Phones_and_Accessories",
    "Digital_Music",
    "Electronics",
    "Grocery_and_Gourmet_Food",
    "Handmade_Products",
    "Health",
    "Industrial_and_Scientific",
    "Kindle_Store",
    "Movies_and_TV",
    "Musical_Instruments",
    "Office_Products",
    "Patio_Lawn_and_Garden",
    "Pet_Supplies",
    "Software",
    "Sports_and_Outdoors",
    "Toys_and_Games",
    "Video_Games",
]

# Force offline to avoid any downloads
os.environ["HF_HUB_OFFLINE"] = "1"
os.environ["TRANSFORMERS_OFFLINE"] = "1"
os.environ["HF_DATASETS_OFFLINE"] = "1"

# Gate router paths (from MoLoRA_Gating_lllm (1).ipynb)
GATE_CKPT_DIR = Path("/root/autodl-tmp/cache/llmRouterMLPPlus")
GATE_ROUTER_PATH = GATE_CKPT_DIR / "gate_router_mlp_plus.pt"
GATE_LABEL_MAP_PATH = GATE_CKPT_DIR / "gate_label_mapping.json"
ST_MAX_LENGTH = 256
ST_MODEL_NAME = "/root/autodl-tmp/cache/hub/models--sentence-transformers--all-MiniLM-L6-v2/snapshots/c9745ed1d9f207416be6d2e6f8de32d1f16199bf"
if not Path(ST_MODEL_NAME).is_dir():
    raise FileNotFoundError(f"ST model path not found: {ST_MODEL_NAME}")
print("[DEBUG] ST_MODEL_NAME:", ST_MODEL_NAME)

TOPK = 3
CONF_THRESHOLD = 0.7  # set >0 to drop low-conf router picks
FALLBACK_CATEGORY = None  # None => first available

# Absolute path templates (CATEGORY will be replaced)
VISION_TEMPLATE = "/root/autodl-tmp/lavic/src/out_distilled/{category}/lora_adapter_best"
LM_TEMPLATE = "/root/autodl-tmp/lavic/src/out_finetuned/{category}/prompt_tuning/trained_lora_adapter"

print("[DEBUG] Offline mode: HF_HUB_OFFLINE=1, TRANSFORMERS_OFFLINE=1")
print("[DEBUG] VISION_TEMPLATE:", VISION_TEMPLATE)
print("[DEBUG] LM_TEMPLATE:", LM_TEMPLATE)

class SimpleRouter(nn.Module):
    def __init__(self, hidden_size, num_adapters):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(hidden_size, num_adapters))
    def forward(self, x):
        return self.net(x)

if BATCH_SIZE != 1:
    raise ValueError("Top-3 router evaluation assumes BATCH_SIZE=1 (image placement).")

if not GATE_ROUTER_PATH.exists() or not GATE_LABEL_MAP_PATH.exists():
    raise FileNotFoundError("Missing gate router or label mapping.")

with GATE_LABEL_MAP_PATH.open("r", encoding="utf-8") as f:
    label_map = json.load(f)
sorted_indices = sorted(int(i) for i in label_map.keys())
gate_categories = [label_map[str(i)] for i in sorted_indices]
print("[DEBUG] Router categories:", len(gate_categories))

st_model = SentenceTransformer(ST_MODEL_NAME, device=DEVICE, cache_folder=os.environ.get("HF_HOME"), local_files_only=True)
st_model.max_seq_length = ST_MAX_LENGTH
embed_dim = st_model.get_sentence_embedding_dimension()
print("[DEBUG] ST model dim:", embed_dim)

class RouterMLPPlus(nn.Module):
    def __init__(self, hidden_size, num_adapters, widths=(512, 256), dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(hidden_size, widths[0]),
            nn.LayerNorm(widths[0]),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(widths[0], widths[1]),
            nn.LayerNorm(widths[1]),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(widths[1], num_adapters)
        )
    def forward(self, x):
        return self.net(x)

ROUTER_TYPE = "RouterMLPPlus"  # or "SimpleRouter"
state = torch.load(GATE_ROUTER_PATH, map_location=DEVICE)
if ROUTER_TYPE == "RouterMLPPlus":
    router = RouterMLPPlus(hidden_size=embed_dim, num_adapters=len(gate_categories)).to(DEVICE).to(DTYPE)
else:
    router = SimpleRouter(hidden_size=embed_dim, num_adapters=len(gate_categories)).to(DEVICE).to(DTYPE)
router.load_state_dict(state)
router.eval()
print(f"[DEBUG] Router loaded: {ROUTER_TYPE}")

# Discover adapter pairs per category
category_to_adapters = {}
for cat in gate_categories:
    v_dir = Path(VISION_TEMPLATE.format(category=cat))
    l_dir = Path(LM_TEMPLATE.format(category=cat))
    if v_dir.is_dir() and l_dir.is_dir():
        category_to_adapters[cat] = {
            "vision": str(v_dir),
            "lm": str(l_dir),
        }
print("[DEBUG] Adapter pairs found:", len(category_to_adapters))

if not category_to_adapters:
    raise RuntimeError("No valid (vision+lm) adapter pairs found.")

if FALLBACK_CATEGORY is None or FALLBACK_CATEGORY not in category_to_adapters:
    FALLBACK_CATEGORY = sorted(category_to_adapters.keys())[0]
print("[DEBUG] FALLBACK_CATEGORY:", FALLBACK_CATEGORY)


def evaluate_category(category: str) -> None:
    global CATEGORY, ITEM_META_PATH, IMAGE_DIR, TEST_DATA_PATH, FINETUNE_OUTPUT_DIR

    CATEGORY = category
    ITEM_META_PATH = DATA_ROOT / f"item2meta_train_{CATEGORY}.with_desc.json"
    IMAGE_DIR = DATA_ROOT / "train_images" / CATEGORY
    TEST_DATA_PATH = DATA_ROOT / CATEGORY / "test.jsonl"
    FINETUNE_OUTPUT_DIR = f"./plora_test_out_v3/{CATEGORY}"

    print("\n========================================")
    print(f"[DEBUG] Start category: {CATEGORY}")
    print("ITEM_META_PATH:", ITEM_META_PATH)
    print("IMAGE_DIR:", IMAGE_DIR)
    print("TEST_DATA_PATH:", TEST_DATA_PATH)
    print("FINETUNE_OUTPUT_DIR:", FINETUNE_OUTPUT_DIR)
    print("========================================")

    item_meta = load_item_meta(ITEM_META_PATH)
    test_data = load_jsonl(TEST_DATA_PATH)
    test_ds = RecommendationDataset(test_data, item_meta, IMAGE_DIR, CANDIDATE_TYPE)
    collator = DataCollatorForLLaVA(processor, tokenizer, MAX_LENGTH)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=lambda x: x)

    # Pass 1: route top-3 for each sample
    sample_top3 = []
    category_to_samples = {cat: [] for cat in category_to_adapters.keys()}
    empty_after_filter = 0
    fallback_top1 = 0
    fallback_fixed = 0

    with torch.no_grad():
        for batch_idx, batch in enumerate(tqdm(test_loader, desc=f"Routing {CATEGORY}", unit="batch"), 1):
            entry_idx = batch[0].get('entry_idx', None)
            if entry_idx is None:
                raise ValueError("Missing entry_idx in batch.")
            context_text = test_data[entry_idx].get('context', '').strip()
            emb = st_model.encode([context_text], convert_to_tensor=True, device=DEVICE, show_progress_bar=False)
            emb = emb.to(DTYPE)
            logits = router(emb)
            probs = torch.softmax(logits, dim=-1)
            top_probs, top_idx = torch.topk(probs, k=min(TOPK, probs.shape[-1]), dim=-1)

            top_cats = []
            top_confs = []
            top1_idx = int(top_idx[0, 0].item())
            top1_cat = label_map.get(str(top1_idx), None)
            for i in range(top_idx.shape[-1]):
                idx = int(top_idx[0, i].item())
                cat = label_map.get(str(idx), None)
                conf = float(top_probs[0, i].item())
                if conf < CONF_THRESHOLD or cat not in category_to_adapters:
                    continue
                top_cats.append(cat)
                top_confs.append(conf)

            if not top_cats:
                empty_after_filter += 1
                if top1_cat in category_to_adapters:
                    top_cats = [top1_cat]
                    top_confs = [float(top_probs[0, 0].item())]
                    fallback_top1 += 1
                else:
                    top_cats = [FALLBACK_CATEGORY]
                    top_confs = [1.0]
                    fallback_fixed += 1

            sample_top3.append({
                "top_cats": top_cats,
                "top_confs": top_confs,
            })
            for cat in top_cats:
                category_to_samples[cat].append(entry_idx)

    print(f"[DEBUG] Routing complete for {CATEGORY}. Samples: {len(sample_top3)}")
    print(f"[DEBUG] empty_after_filter={empty_after_filter} fallback_top1={fallback_top1} fallback_fixed={fallback_fixed}")

    # Prepare aggregation containers
    recommended_ids = [None] * len(test_data)
    responses = [""] * len(test_data)
    ground_truths = [entry.get('gt_items', []) for entry in test_data]
    selected_categories = [None] * len(test_data)
    selected_confs = [None] * len(test_data)
    selected_top3 = [None] * len(test_data)
    selected_top3_confs = [None] * len(test_data)
    selected_top3_recs = [None] * len(test_data)

    score_maps = [None] * len(test_data)
    resp_maps = [None] * len(test_data)

    # Pass 2: process per category (load once)
    for cat, sample_ids in category_to_samples.items():
        if not sample_ids:
            continue

        v_dir = category_to_adapters[cat]["vision"]
        l_dir = category_to_adapters[cat]["lm"]
        print(f"[DEBUG] Load model for {cat}:\n  vision={v_dir}\n  lm={l_dir}\n  samples={len(sample_ids)}")

        base = LlavaNextForConditionalGeneration.from_pretrained(
            BASE_MODEL_NAME,
            torch_dtype=DTYPE,
            local_files_only=True,
        ).to(DEVICE)

        # merge vision adapter
        base = PeftModel.from_pretrained(base, v_dir, local_files_only=True)
        base = base.merge_and_unload()
        base = base.to(DEVICE)

        # apply LM adapter
        model = PeftModel.from_pretrained(base, l_dir, local_files_only=True).to(DEVICE)
        model.eval()

        with torch.no_grad():
            for entry_idx in tqdm(sample_ids, desc=f"Evaluating {CATEGORY}->{cat}", unit="sample"):
                batch = [test_ds[entry_idx]]
                inputs = collator(batch)
                input_ids = inputs['input_ids'].to(DEVICE)
                attention_mask = inputs['attention_mask'].to(DEVICE)
                image_token_mask = inputs['image_token_mask'].to(DEVICE)
                images = inputs['images']

                inputs_embeds = model.language_model.get_input_embeddings()(input_ids)
                if images is not None:
                    images = images.to(DEVICE, dtype=DTYPE)
                    if images.dim() == 5:
                        b_img, num_views, c, h, w = images.shape
                        images_reshaped = images.view(-1, c, h, w)
                    else:
                        num_views = 1
                        images_reshaped = images
                    vision_outputs = model.vision_tower(images_reshaped, return_dict=False)
                    cls_states = vision_outputs[0][:, 0, :]
                    total_patches = cls_states.shape[0]
                    expected_images = input_ids.size(0) * num_views
                    if total_patches > expected_images:
                        patches_per_image = total_patches // expected_images
                        if total_patches % expected_images == 0 and patches_per_image > 1:
                            cls_states = cls_states.view(expected_images, patches_per_image, -1).mean(dim=1)
                    cls_states = cls_states.reshape(input_ids.size(0), num_views, -1)
                    cls_states = model.multi_modal_projector(cls_states)
                    for b_idx in range(input_ids.size(0)):
                        positions = torch.nonzero(image_token_mask[b_idx], as_tuple=False).squeeze(-1)
                        pos_count = min(len(positions), num_views)
                        for i in range(pos_count):
                            col = positions[i].item()
                            inputs_embeds[b_idx, col, :] = cls_states[b_idx, i, :]

                generated_ids = model.generate(
                    inputs_embeds=inputs_embeds,
                    attention_mask=attention_mask,
                    max_new_tokens=10,
                    num_beams=1,
                    do_sample=False,
                    pad_token_id=tokenizer.pad_token_id,
                )
                generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
                match = re.findall(r'\bB[A-Z0-9]{9}\b', generated_texts[0].strip())
                rec_id = match[0][:10] if match else None

                # init per-sample aggregation
                if score_maps[entry_idx] is None:
                    score_maps[entry_idx] = {}
                    resp_maps[entry_idx] = {}
                    selected_top3[entry_idx] = sample_top3[entry_idx]["top_cats"]
                    selected_top3_confs[entry_idx] = sample_top3[entry_idx]["top_confs"]
                    selected_categories[entry_idx] = sample_top3[entry_idx]["top_cats"][0]
                    selected_confs[entry_idx] = sample_top3[entry_idx]["top_confs"][0]

                # weight by router confidence for this category
                top_cats = sample_top3[entry_idx]["top_cats"]
                top_confs = sample_top3[entry_idx]["top_confs"]
                conf = top_confs[top_cats.index(cat)] if cat in top_cats else 0.0

                score_maps[entry_idx][rec_id] = score_maps[entry_idx].get(rec_id, 0.0) + conf
                if rec_id not in resp_maps[entry_idx]:
                    resp_maps[entry_idx][rec_id] = generated_texts[0]

        del model
        del base
        torch.cuda.empty_cache()

    # Final fusion per sample
    for i in range(len(test_data)):
        if score_maps[i] is None or not score_maps[i]:
            recommended_ids[i] = None
            responses[i] = ""
            selected_top3_recs[i] = []
            continue

        best_rec_id = max(score_maps[i].items(), key=lambda kv: kv[1])[0]
        recommended_ids[i] = best_rec_id
        responses[i] = resp_maps[i].get(best_rec_id, "")
        selected_top3_recs[i] = [None] * len(selected_top3[i])

    recall = evaluate_recall_at_k(recommended_ids, ground_truths, k=1)
    print(f"[Test] Recall@1 ({CATEGORY}): {recall:.4f}")

    model_key = 'st' if CANDIDATE_TYPE == 'candidates_st' else 'gpt_large'
    if model_key == 'st':
        recommended_field = 'recommended_st'
        response_field = 'response_st'
    else:
        recommended_field = f"recommended_{model_key}"
        response_field = f"response_{model_key}"

    out_file_name = f"test_results_{CANDIDATE_TYPE}_router_top3_vl.jsonl"
    output_file_path = Path(FINETUNE_OUTPUT_DIR) / out_file_name
    output_file_path.parent.mkdir(parents=True, exist_ok=True)

    with output_file_path.open('w', encoding='utf-8') as f:
        for entry, rec_id, resp, sel_cat, sel_conf, top3, top3_confs in zip(
            test_data, recommended_ids, responses, selected_categories, selected_confs,
            selected_top3, selected_top3_confs
        ):
            entry[recommended_field] = rec_id
            entry[response_field] = resp
            entry['selected_category'] = sel_cat
            entry['selected_confidence'] = sel_conf
            entry['router_top3'] = top3
            entry['router_top3_confidences'] = top3_confs
            json.dump(entry, f, ensure_ascii=False)
            f.write('\n')

    validity, invalid_entries, total_count = check_validity(output_file_path, model_key)
    print(f"Validity@1 ({CATEGORY}): {validity:.4f}, invalid entries: {len(invalid_entries)} / {total_count}")
    print(f"Test details saved to {output_file_path}")


[DEBUG] ST_MODEL_NAME: /root/autodl-tmp/cache/hub/models--sentence-transformers--all-MiniLM-L6-v2/snapshots/c9745ed1d9f207416be6d2e6f8de32d1f16199bf
[DEBUG] Offline mode: HF_HUB_OFFLINE=1, TRANSFORMERS_OFFLINE=1
[DEBUG] VISION_TEMPLATE: /root/autodl-tmp/lavic/src/out_distilled/{category}/lora_adapter_best
[DEBUG] LM_TEMPLATE: /root/autodl-tmp/lavic/src/out_finetuned/{category}/prompt_tuning/trained_lora_adapter
[DEBUG] Router categories: 26
[DEBUG] ST model dim: 384
[DEBUG] Router loaded: RouterMLPPlus
[DEBUG] Adapter pairs found: 26
[DEBUG] FALLBACK_CATEGORY: Appliances


In [6]:
# Debug router confidence distribution for a single category
import numpy as np

debug_category = "Books"  # change if needed
max_samples = 200  # limit for quick debug

if 'category_to_adapters' not in globals():
    raise RuntimeError("category_to_adapters not found. Run the setup cell first.")

ITEM_META_PATH = DATA_ROOT / f"item2meta_train_{debug_category}.with_desc.json"
TEST_DATA_PATH = DATA_ROOT / debug_category / "test.jsonl"

test_data = load_jsonl(TEST_DATA_PATH)

pmax_list = []
empty_after_filter = 0

with torch.no_grad():
    for i, entry in enumerate(test_data[:max_samples]):
        context_text = entry.get('context', '').strip()
        emb = st_model.encode([context_text], convert_to_tensor=True, device=DEVICE, show_progress_bar=False)
        emb = emb.to(DTYPE)
        logits = router(emb)
        probs = torch.softmax(logits, dim=-1)
        pmax, imax = probs.max(dim=-1)
        pmax_list.append(float(pmax.item()))

        top_probs, top_idx = torch.topk(probs, k=min(TOPK, probs.shape[-1]), dim=-1)
        top_cats = []
        for j in range(top_idx.shape[-1]):
            idx = int(top_idx[0, j].item())
            cat = label_map.get(str(idx), None)
            conf = float(top_probs[0, j].item())
            if conf < CONF_THRESHOLD or cat not in category_to_adapters:
                continue
            top_cats.append(cat)
        if not top_cats:
            empty_after_filter += 1

pmax_arr = np.array(pmax_list, dtype=float)
print(f"[DEBUG] category={debug_category} samples={len(pmax_arr)}")
print(f"[DEBUG] pmax mean={pmax_arr.mean():.4f} median={np.median(pmax_arr):.4f} min={pmax_arr.min():.4f} max={pmax_arr.max():.4f}")
print(f"[DEBUG] empty_after_filter={empty_after_filter}/{len(pmax_arr)} (CONF_THRESHOLD={CONF_THRESHOLD}, TOPK={TOPK})")
print("[DEBUG] pmax histogram bins=10:")
counts, bins = np.histogram(pmax_arr, bins=10, range=(0.0, 1.0))
for c, b0, b1 in zip(counts, bins[:-1], bins[1:]):
    print(f"  {b0:.1f}-{b1:.1f}: {c}")

[DEBUG] category=Books samples=200
[DEBUG] pmax mean=0.7801 median=0.8191 min=0.3184 max=0.9136
[DEBUG] empty_after_filter=10/200 (CONF_THRESHOLD=0.5, TOPK=3)
[DEBUG] pmax histogram bins=10:
  0.0-0.1: 0
  0.1-0.2: 0
  0.2-0.3: 0
  0.3-0.4: 7
  0.4-0.5: 3
  0.5-0.6: 9
  0.6-0.7: 18
  0.7-0.8: 40
  0.8-0.9: 116
  0.9-1.0: 7


In [11]:
evaluate_category("Automotive")


[DEBUG] Start category: Automotive
ITEM_META_PATH: /root/autodl-tmp/lavic/data/item2meta_train_Automotive.with_desc.json
IMAGE_DIR: /root/autodl-tmp/lavic/data/train_images/Automotive
TEST_DATA_PATH: /root/autodl-tmp/lavic/data/Automotive/test.jsonl
FINETUNE_OUTPUT_DIR: ./plora_test_out_v3/Automotive


Routing Automotive: 100%|██████████| 152/152 [00:02<00:00, 73.17batch/s]

[DEBUG] Routing complete for Automotive. Samples: 152
[DEBUG] empty_after_filter=22 fallback_top1=22 fallback_fixed=0
[DEBUG] Load model for Arts_Crafts_and_Sewing:
  vision=/root/autodl-tmp/lavic/src/out_distilled/Arts_Crafts_and_Sewing/lora_adapter_best
  lm=/root/autodl-tmp/lavic/src/out_finetuned/Arts_Crafts_and_Sewing/prompt_tuning/trained_lora_adapter
  samples=2


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating Automotive->Arts_Crafts_and_Sewing: 100%|██████████| 2/2 [00:01<00:00,  1.51sample/s]

[DEBUG] Load model for Automotive:
  vision=/root/autodl-tmp/lavic/src/out_distilled/Automotive/lora_adapter_best
  lm=/root/autodl-tmp/lavic/src/out_finetuned/Automotive/prompt_tuning/trained_lora_adapter
  samples=138


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [50]:
evaluate_category("all_beauty")


[DEBUG] Start category: all_beauty
ITEM_META_PATH: /root/autodl-tmp/lavic/data/item2meta_train_all_beauty.with_desc.json
IMAGE_DIR: /root/autodl-tmp/lavic/data/train_images/all_beauty
TEST_DATA_PATH: /root/autodl-tmp/lavic/data/all_beauty/test.jsonl
FINETUNE_OUTPUT_DIR: ./plora_test_out_v3/all_beauty


Routing all_beauty: 100%|██████████| 772/772 [00:10<00:00, 74.52batch/s]

[DEBUG] Routing complete for all_beauty. Samples: 772
[DEBUG] Load model for Arts_Crafts_and_Sewing:
  vision=/root/autodl-tmp/lavic/src/out_distilled/Arts_Crafts_and_Sewing/lora_adapter_best
  lm=/root/autodl-tmp/lavic/src/out_finetuned/Arts_Crafts_and_Sewing/prompt_tuning/trained_lora_adapter
  samples=2


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating all_beauty->Arts_Crafts_and_Sewing: 100%|██████████| 2/2 [00:01<00:00,  1.48sample/s]

...

Evaluating all_beauty->amazon_fashion: 100%|██████████| 38/38 [00:23<00:00,  1.59sample/s]

[DEBUG] Load model for amazon_home:
  vision=/root/autodl-tmp/lavic/src/out_distilled/amazon_home/lora_adapter_best
  lm=/root/autodl-tmp/lavic/src/out_finetuned/amazon_home/prompt_tuning/trained_lora_adapter
  samples=29


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating all_beauty->amazon_home: 100%|██████████| 29/29 [00:19<00:00,  1.52sample/s]

[Test] Recall@1 (all_beauty): 0.1000
Validity@1 (all_beauty): 0.9909, invalid entries: 7 / 772
Test details saved to plora_test_out_v3/all_beauty/test_results_candidates_st_router_top3_vl.jsonl


In [51]:
evaluate_category("amazon_fashion")


[DEBUG] Start category: amazon_fashion
ITEM_META_PATH: /root/autodl-tmp/lavic/data/item2meta_train_amazon_fashion.with_desc.json
IMAGE_DIR: /root/autodl-tmp/lavic/data/train_images/amazon_fashion
TEST_DATA_PATH: /root/autodl-tmp/lavic/data/amazon_fashion/test.jsonl
FINETUNE_OUTPUT_DIR: ./plora_test_out_v3/amazon_fashion


Routing amazon_fashion: 100%|██████████| 826/826 [00:09<00:00, 89.50batch/s]


[DEBUG] Routing complete for amazon_fashion. Samples: 826
[DEBUG] Load model for Arts_Crafts_and_Sewing:
  vision=/root/autodl-tmp/lavic/src/out_distilled/Arts_Crafts_and_Sewing/lora_adapter_best
  lm=/root/autodl-tmp/lavic/src/out_finetuned/Arts_Crafts_and_Sewing/prompt_tuning/trained_lora_adapter
  samples=3


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating amazon_fashion->Arts_Crafts_and_Sewing: 100%|██████████| 3/3 [00:01<00:00,  1.64sample/s]

...

Evaluating amazon_fashion->amazon_fashion: 100%|██████████| 658/658 [06:30<00:00,  1.69sample/s]

[DEBUG] Load model for amazon_home:
  vision=/root/autodl-tmp/lavic/src/out_distilled/amazon_home/lora_adapter_best
  lm=/root/autodl-tmp/lavic/src/out_finetuned/amazon_home/prompt_tuning/trained_lora_adapter
  samples=41


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating amazon_fashion->amazon_home: 100%|██████████| 41/41 [00:25<00:00,  1.60sample/s]

[Test] Recall@1 (amazon_fashion): 0.1072
Validity@1 (amazon_fashion): 0.9915, invalid entries: 7 / 826
Test details saved to plora_test_out_v3/amazon_fashion/test_results_candidates_st_router_top3_vl.jsonl


In [52]:
evaluate_category("amazon_home")


[DEBUG] Start category: amazon_home
ITEM_META_PATH: /root/autodl-tmp/lavic/data/item2meta_train_amazon_home.with_desc.json
IMAGE_DIR: /root/autodl-tmp/lavic/data/train_images/amazon_home
TEST_DATA_PATH: /root/autodl-tmp/lavic/data/amazon_home/test.jsonl
FINETUNE_OUTPUT_DIR: ./plora_test_out_v3/amazon_home


Routing amazon_home: 100%|██████████| 372/372 [00:04<00:00, 84.65batch/s]


[DEBUG] Routing complete for amazon_home. Samples: 372
[DEBUG] Load model for Appliances:
  vision=/root/autodl-tmp/lavic/src/out_distilled/Appliances/lora_adapter_best
  lm=/root/autodl-tmp/lavic/src/out_finetuned/Appliances/prompt_tuning/trained_lora_adapter
  samples=3


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating amazon_home->Appliances: 100%|██████████| 3/3 [00:01<00:00,  1.57sample/s]

...

Evaluating amazon_home->amazon_fashion: 100%|██████████| 20/20 [00:12<00:00,  1.57sample/s]

[DEBUG] Load model for amazon_home:
  vision=/root/autodl-tmp/lavic/src/out_distilled/amazon_home/lora_adapter_best
  lm=/root/autodl-tmp/lavic/src/out_finetuned/amazon_home/prompt_tuning/trained_lora_adapter
  samples=218


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating amazon_home->amazon_home: 100%|██████████| 218/218 [02:17<00:00,  1.58sample/s]

[Test] Recall@1 (amazon_home): 0.1990
Validity@1 (amazon_home): 1.0000, invalid entries: 0 / 372
Test details saved to plora_test_out_v3/amazon_home/test_results_candidates_st_router_top3_vl.jsonl


In [58]:
#run with CONF_THRESHOLD = 0.9
evaluate_category("Books")


[DEBUG] Start category: Books
ITEM_META_PATH: /root/autodl-tmp/lavic/data/item2meta_train_Books.with_desc.json
IMAGE_DIR: /root/autodl-tmp/lavic/data/train_images/Books
TEST_DATA_PATH: /root/autodl-tmp/lavic/data/Books/test.jsonl
FINETUNE_OUTPUT_DIR: ./plora_test_out_v3/Books


Routing Books: 100%|██████████| 472/472 [00:06<00:00, 71.46batch/s]

[DEBUG] Routing complete for Books. Samples: 472
[DEBUG] Load model for Books:
  vision=/root/autodl-tmp/lavic/src/out_distilled/Books/lora_adapter_best
  lm=/root/autodl-tmp/lavic/src/out_finetuned/Books/prompt_tuning/trained_lora_adapter
  samples=448


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating Books->Books: 100%|██████████| 448/448 [05:36<00:00,  1.33sample/s]

...

Evaluating Books->Patio_Lawn_and_Garden: 100%|██████████| 1/1 [00:00<00:00,  1.19sample/s]

[DEBUG] Load model for amazon_fashion:
  vision=/root/autodl-tmp/lavic/src/out_distilled/amazon_fashion/lora_adapter_best
  lm=/root/autodl-tmp/lavic/src/out_finetuned/amazon_fashion/prompt_tuning/trained_lora_adapter
  samples=1


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating Books->amazon_fashion: 100%|██████████| 1/1 [00:00<00:00,  1.14sample/s]

[Test] Recall@1 (Books): 0.0318
Validity@1 (Books): 0.2881, invalid entries: 336 / 472
Test details saved to plora_test_out_v3/Books/test_results_candidates_st_router_top3_vl.jsonl


In [59]:
#run with CONF_THRESHOLD = 0.9
evaluate_category("Toys_and_Games")


[DEBUG] Start category: Toys_and_Games
ITEM_META_PATH: /root/autodl-tmp/lavic/data/item2meta_train_Toys_and_Games.with_desc.json
IMAGE_DIR: /root/autodl-tmp/lavic/data/train_images/Toys_and_Games
TEST_DATA_PATH: /root/autodl-tmp/lavic/data/Toys_and_Games/test.jsonl
FINETUNE_OUTPUT_DIR: ./plora_test_out_v3/Toys_and_Games


Routing Toys_and_Games: 100%|██████████| 192/192 [00:03<00:00, 58.45batch/s]


[DEBUG] Routing complete for Toys_and_Games. Samples: 192
[DEBUG] Load model for Automotive:
  vision=/root/autodl-tmp/lavic/src/out_distilled/Automotive/lora_adapter_best
  lm=/root/autodl-tmp/lavic/src/out_finetuned/Automotive/prompt_tuning/trained_lora_adapter
  samples=1


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating Toys_and_Games->Automotive: 100%|██████████| 1/1 [00:00<00:00,  1.10sample/s]

...

Evaluating Toys_and_Games->Video_Games: 100%|██████████| 4/4 [00:02<00:00,  1.35sample/s]

[DEBUG] Load model for amazon_home:
  vision=/root/autodl-tmp/lavic/src/out_distilled/amazon_home/lora_adapter_best
  lm=/root/autodl-tmp/lavic/src/out_finetuned/amazon_home/prompt_tuning/trained_lora_adapter
  samples=3


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating Toys_and_Games->amazon_home: 100%|██████████| 3/3 [00:02<00:00,  1.20sample/s]

[Test] Recall@1 (Toys_and_Games): 0.3177
Validity@1 (Toys_and_Games): 1.0000, invalid entries: 0 / 192
Test details saved to plora_test_out_v3/Toys_and_Games/test_results_candidates_st_router_top3_vl.jsonl


In [60]:
#run with CONF_THRESHOLD = 0.9
evaluate_category("Appliances")


[DEBUG] Start category: Appliances
ITEM_META_PATH: /root/autodl-tmp/lavic/data/item2meta_train_Appliances.with_desc.json
IMAGE_DIR: /root/autodl-tmp/lavic/data/train_images/Appliances
TEST_DATA_PATH: /root/autodl-tmp/lavic/data/Appliances/test.jsonl
FINETUNE_OUTPUT_DIR: ./plora_test_out_v3/Appliances


Routing Appliances: 100%|██████████| 230/230 [00:03<00:00, 72.59batch/s]


[DEBUG] Routing complete for Appliances. Samples: 230
[DEBUG] Load model for Appliances:
  vision=/root/autodl-tmp/lavic/src/out_distilled/Appliances/lora_adapter_best
  lm=/root/autodl-tmp/lavic/src/out_finetuned/Appliances/prompt_tuning/trained_lora_adapter
  samples=222


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating Appliances->Appliances: 100%|██████████| 222/222 [02:44<00:00,  1.35sample/s]

...

Evaluating Appliances->Software: 100%|██████████| 1/1 [00:00<00:00,  1.03sample/s]

[DEBUG] Load model for amazon_home:
  vision=/root/autodl-tmp/lavic/src/out_distilled/amazon_home/lora_adapter_best
  lm=/root/autodl-tmp/lavic/src/out_finetuned/amazon_home/prompt_tuning/trained_lora_adapter
  samples=6


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating Appliances->amazon_home: 100%|██████████| 6/6 [00:03<00:00,  1.57sample/s]

[Test] Recall@1 (Appliances): 0.3522
Validity@1 (Appliances): 0.9957, invalid entries: 1 / 230
Test details saved to plora_test_out_v3/Appliances/test_results_candidates_st_router_top3_vl.jsonl


In [62]:
#run with CONF_THRESHOLD = 0.5
evaluate_category("Books")


[DEBUG] Start category: Books
ITEM_META_PATH: /root/autodl-tmp/lavic/data/item2meta_train_Books.with_desc.json
IMAGE_DIR: /root/autodl-tmp/lavic/data/train_images/Books
TEST_DATA_PATH: /root/autodl-tmp/lavic/data/Books/test.jsonl
FINETUNE_OUTPUT_DIR: ./plora_test_out_v3/Books


Routing Books: 100%|██████████| 472/472 [00:06<00:00, 72.24batch/s]

[DEBUG] Routing complete for Books. Samples: 472
[DEBUG] Load model for Books:
  vision=/root/autodl-tmp/lavic/src/out_distilled/Books/lora_adapter_best
  lm=/root/autodl-tmp/lavic/src/out_finetuned/Books/prompt_tuning/trained_lora_adapter
  samples=448


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating Books->Books: 100%|██████████| 448/448 [05:59<00:00,  1.25sample/s]

...

Evaluating Books->Patio_Lawn_and_Garden: 100%|██████████| 1/1 [00:00<00:00,  1.21sample/s]

[DEBUG] Load model for amazon_fashion:
  vision=/root/autodl-tmp/lavic/src/out_distilled/amazon_fashion/lora_adapter_best
  lm=/root/autodl-tmp/lavic/src/out_finetuned/amazon_fashion/prompt_tuning/trained_lora_adapter
  samples=1


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating Books->amazon_fashion: 100%|██████████| 1/1 [00:00<00:00,  1.07sample/s]

[Test] Recall@1 (Books): 0.0318
Validity@1 (Books): 0.2881, invalid entries: 336 / 472
Test details saved to plora_test_out_v3/Books/test_results_candidates_st_router_top3_vl.jsonl


In [63]:
#run with CONF_THRESHOLD = 0.5
evaluate_category("Toys_and_Games")


[DEBUG] Start category: Toys_and_Games
ITEM_META_PATH: /root/autodl-tmp/lavic/data/item2meta_train_Toys_and_Games.with_desc.json
IMAGE_DIR: /root/autodl-tmp/lavic/data/train_images/Toys_and_Games
TEST_DATA_PATH: /root/autodl-tmp/lavic/data/Toys_and_Games/test.jsonl
FINETUNE_OUTPUT_DIR: ./plora_test_out_v3/Toys_and_Games


Routing Toys_and_Games: 100%|██████████| 192/192 [00:03<00:00, 53.23batch/s]


[DEBUG] Routing complete for Toys_and_Games. Samples: 192
[DEBUG] Load model for Automotive:
  vision=/root/autodl-tmp/lavic/src/out_distilled/Automotive/lora_adapter_best
  lm=/root/autodl-tmp/lavic/src/out_finetuned/Automotive/prompt_tuning/trained_lora_adapter
  samples=1


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating Toys_and_Games->Automotive: 100%|██████████| 1/1 [00:00<00:00,  1.11sample/s]

...

Evaluating Toys_and_Games->Video_Games: 100%|██████████| 4/4 [00:02<00:00,  1.60sample/s]

[DEBUG] Load model for amazon_home:
  vision=/root/autodl-tmp/lavic/src/out_distilled/amazon_home/lora_adapter_best
  lm=/root/autodl-tmp/lavic/src/out_finetuned/amazon_home/prompt_tuning/trained_lora_adapter
  samples=3


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating Toys_and_Games->amazon_home: 100%|██████████| 3/3 [00:02<00:00,  1.16sample/s]

[Test] Recall@1 (Toys_and_Games): 0.3177
Validity@1 (Toys_and_Games): 1.0000, invalid entries: 0 / 192
Test details saved to plora_test_out_v3/Toys_and_Games/test_results_candidates_st_router_top3_vl.jsonl


In [64]:
#run with CONF_THRESHOLD = 0.5
evaluate_category("Appliances")


[DEBUG] Start category: Appliances
ITEM_META_PATH: /root/autodl-tmp/lavic/data/item2meta_train_Appliances.with_desc.json
IMAGE_DIR: /root/autodl-tmp/lavic/data/train_images/Appliances
TEST_DATA_PATH: /root/autodl-tmp/lavic/data/Appliances/test.jsonl
FINETUNE_OUTPUT_DIR: ./plora_test_out_v3/Appliances


Routing Appliances: 100%|██████████| 230/230 [00:03<00:00, 63.13batch/s]

[DEBUG] Routing complete for Appliances. Samples: 230
[DEBUG] Load model for Appliances:
  vision=/root/autodl-tmp/lavic/src/out_distilled/Appliances/lora_adapter_best
  lm=/root/autodl-tmp/lavic/src/out_finetuned/Appliances/prompt_tuning/trained_lora_adapter
  samples=222


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating Appliances->Appliances: 100%|██████████| 222/222 [02:57<00:00,  1.25sample/s]

...

Evaluating Appliances->Software: 100%|██████████| 1/1 [00:00<00:00,  1.16sample/s]

[DEBUG] Load model for amazon_home:
  vision=/root/autodl-tmp/lavic/src/out_distilled/amazon_home/lora_adapter_best
  lm=/root/autodl-tmp/lavic/src/out_finetuned/amazon_home/prompt_tuning/trained_lora_adapter
  samples=6


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating Appliances->amazon_home: 100%|██████████| 6/6 [00:05<00:00,  1.14sample/s]

[Test] Recall@1 (Appliances): 0.3522
Validity@1 (Appliances): 0.9957, invalid entries: 1 / 230
Test details saved to plora_test_out_v3/Appliances/test_results_candidates_st_router_top3_vl.jsonl


In [ ]:
#amz are in new_fin

In [7]:
# Threshold sweep for Books
import numpy as np
from pathlib import Path

if 'threshold_sweeps' not in globals():
    threshold_sweeps = {}

category = "Books"
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
rows = []

for tau in thresholds:
    CONF_THRESHOLD = tau
    print(f"[SWEEP] category={category} CONF_THRESHOLD={CONF_THRESHOLD:.1f}")
    evaluate_category(category)

    test_data_path = DATA_ROOT / category / "test.jsonl"
    test_data = load_jsonl(test_data_path)
    ground_truths = [entry.get('gt_items', []) for entry in test_data]

    model_key = 'st' if CANDIDATE_TYPE == 'candidates_st' else 'gpt_large'
    recommended_field = 'recommended_st' if model_key == 'st' else f"recommended_{model_key}"
    out_file = Path(f"./plora_test_out_v3/{category}") / f"test_results_{CANDIDATE_TYPE}_router_top3_vl.jsonl"

    results = load_jsonl(out_file)
    recommended_ids = [entry.get(recommended_field) for entry in results]
    recall = evaluate_recall_at_k(recommended_ids, ground_truths, k=1)
    validity, _, _ = check_validity(out_file, model_key)

    rows.append({
        'category': category,
        'tau': tau,
        'recall': recall,
        'validity': validity,
    })

threshold_sweeps[category] = rows
print(f"[SWEEP] done for {category}: {len(rows)} thresholds")

[SWEEP] category=Books CONF_THRESHOLD=0.3

[DEBUG] Start category: Books
ITEM_META_PATH: /root/autodl-tmp/lavic/data/item2meta_train_Books.with_desc.json
IMAGE_DIR: /root/autodl-tmp/lavic/data/train_images/Books
TEST_DATA_PATH: /root/autodl-tmp/lavic/data/Books/test.jsonl
FINETUNE_OUTPUT_DIR: ./plora_test_out_v3/Books


Routing Books: 100%|██████████| 472/472 [00:05<00:00, 87.04batch/s]


[DEBUG] Routing complete for Books. Samples: 472
[DEBUG] Load model for Books:
  vision=/root/autodl-tmp/lavic/src/out_distilled/Books/lora_adapter_best
  lm=/root/autodl-tmp/lavic/src/out_finetuned/Books/prompt_tuning/trained_lora_adapter
  samples=452


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/root/miniconda3/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
Evaluating Books->Books:  52%|█████▏    | 235/452 [02:23<02:12,  1.63sample/s]


ValueError: Unable to create tensor, you should probably activate padding with 'padding=True' to have batched tensors with the same length.

In [ ]:
# Threshold sweep for Toys_and_Games
import numpy as np
from pathlib import Path

if 'threshold_sweeps' not in globals():
    threshold_sweeps = {}

category = "Toys_and_Games"
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
rows = []

for tau in thresholds:
    CONF_THRESHOLD = tau
    print(f"[SWEEP] category={category} CONF_THRESHOLD={CONF_THRESHOLD:.1f}")
    evaluate_category(category)

    test_data_path = DATA_ROOT / category / "test.jsonl"
    test_data = load_jsonl(test_data_path)
    ground_truths = [entry.get('gt_items', []) for entry in test_data]

    model_key = 'st' if CANDIDATE_TYPE == 'candidates_st' else 'gpt_large'
    recommended_field = 'recommended_st' if model_key == 'st' else f"recommended_{model_key}"
    out_file = Path(f"./plora_test_out_v3/{category}") / f"test_results_{CANDIDATE_TYPE}_router_top3_vl.jsonl"

    results = load_jsonl(out_file)
    recommended_ids = [entry.get(recommended_field) for entry in results]
    recall = evaluate_recall_at_k(recommended_ids, ground_truths, k=1)
    validity, _, _ = check_validity(out_file, model_key)

    rows.append({
        'category': category,
        'tau': tau,
        'recall': recall,
        'validity': validity,
    })

threshold_sweeps[category] = rows
print(f"[SWEEP] done for {category}: {len(rows)} thresholds")

In [ ]:
# Threshold sweep for Appliances
import numpy as np
from pathlib import Path

if 'threshold_sweeps' not in globals():
    threshold_sweeps = {}

category = "Appliances"
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
rows = []

for tau in thresholds:
    CONF_THRESHOLD = tau
    print(f"[SWEEP] category={category} CONF_THRESHOLD={CONF_THRESHOLD:.1f}")
    evaluate_category(category)

    test_data_path = DATA_ROOT / category / "test.jsonl"
    test_data = load_jsonl(test_data_path)
    ground_truths = [entry.get('gt_items', []) for entry in test_data]

    model_key = 'st' if CANDIDATE_TYPE == 'candidates_st' else 'gpt_large'
    recommended_field = 'recommended_st' if model_key == 'st' else f"recommended_{model_key}"
    out_file = Path(f"./plora_test_out_v3/{category}") / f"test_results_{CANDIDATE_TYPE}_router_top3_vl.jsonl"

    results = load_jsonl(out_file)
    recommended_ids = [entry.get(recommended_field) for entry in results]
    recall = evaluate_recall_at_k(recommended_ids, ground_truths, k=1)
    validity, _, _ = check_validity(out_file, model_key)

    rows.append({
        'category': category,
        'tau': tau,
        'recall': recall,
        'validity': validity,
    })

threshold_sweeps[category] = rows
print(f"[SWEEP] done for {category}: {len(rows)} thresholds")

In [ ]:
# Summarize sweeps: table + line plot (no pandas)
import matplotlib.pyplot as plt

if 'threshold_sweeps' not in globals() or not threshold_sweeps:
    raise RuntimeError("threshold_sweeps is empty. Run the per-category sweep cells first.")

rows = []
for cat, items in threshold_sweeps.items():
    rows.extend(items)

if not rows:
    raise RuntimeError("No sweep results found.")

# Table (sorted)
rows_sorted = sorted(rows, key=lambda r: (r['category'], r['tau']))
print("category\ttau\trecall\tvalidity")
for r in rows_sorted:
    print(f"{r['category']}\t{r['tau']:.1f}\t{r['recall']:.4f}\t{r['validity']:.4f}")

# Best tau per category (by recall)
best_by_cat = {}
for r in rows_sorted:
    key = r['category']
    if key not in best_by_cat or r['recall'] > best_by_cat[key]['recall']:
        best_by_cat[key] = r
print("\nBest tau per category (by recall):")
print("category\ttau\trecall\tvalidity")
for key in sorted(best_by_cat.keys()):
    r = best_by_cat[key]
    print(f"{r['category']}\t{r['tau']:.1f}\t{r['recall']:.4f}\t{r['validity']:.4f}")

# Line plot: recall vs tau
plt.figure(figsize=(7, 4))
for cat in sorted({r['category'] for r in rows_sorted}):
    sub = [r for r in rows_sorted if r['category'] == cat]
    taus = [r['tau'] for r in sub]
    recalls = [r['recall'] for r in sub]
    plt.plot(taus, recalls, marker='o', label=cat)

plt.title('Recall@1 vs CONF_THRESHOLD')
plt.xlabel('CONF_THRESHOLD (tau)')
plt.ylabel('Recall@1')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()